# Viterbi Heuristic

## Introduction
- The **Viterbi Heuristic** is a simplified version of the **Viterbi Algorithm**, which is used to find the most probable sequence of hidden states in a **Hidden Markov Model (HMM)**. 
- It is typically used in tasks like **part-of-speech tagging**, **speech recognition**, and **machine translation**.

## Key Components of HMM
1. **States**: Hidden states (e.g., parts of speech, phonemes).
2. **Observations**: Observable outputs corresponding to each state (e.g., words, acoustic features).
3. **Transition Probabilities**: Probabilities of moving from one state to another.
4. **Emission Probabilities**: Probabilities of observing a particular output from a state.

## Viterbi Algorithm
The Viterbi algorithm computes the most probable sequence of hidden states for a given sequence of observations.

### Steps:
1. **Initialization**: Calculate the probability of starting in each state for the first observation.
2. **Recursion**: For each subsequent observation, calculate the probability of arriving at each state from every possible previous state.
3. **Termination**: Once the last observation is processed, backtrack to find the best state sequence.

## Viterbi Heuristic
The **Viterbi Heuristic** is a simplified or approximate version of the Viterbi algorithm. It:
- **Limits the search space** to likely states.
- **Prunes unlikely paths**.
- **Uses approximation techniques** to speed up the computation.

### Common Techniques:
- **Pruning**: Discarding unlikely paths early on.
- **Greedy Decisions**: Choosing the most probable state at each step.
- **Beam Search**: Keeping the top **k** most likely paths at each step.

## Example Use Case
### Part-of-Speech Tagging:
- Input: "The cat sleeps."
- Hidden states: "DT" (Determiner), "NN" (Noun), "VBZ" (Verb).
- The Viterbi algorithm finds the most likely sequence of tags for this sentence. The heuristic might simplify by:
  - Keeping only the top **k** most likely sequences.
  - Pruning unlikely tag transitions.




# Understanding Transition and Emission Probabilities for Part-of-Speech Tagging

## 1. Transition Probabilities:
Transition probabilities represent the likelihood of transitioning from one state to another. 

- **Example in POS tagging**: 
  - It tells you how likely it is for a word tagged as **"Determiner" (DT)** to be followed by a word tagged as **"Noun" (NN)**, or for a verb to follow a noun, etc.
  - For instance, we might expect the probability of going from **DT → NN** to be high, while **NN → VBZ** might be less likely.

- **Why We Need Transition Probabilities**:
  - Without transition probabilities, the algorithm wouldn’t have a basis to decide the most probable state transitions between consecutive words in the sequence.
  
## 2. Emission Probabilities:
Emission probabilities represent the likelihood of a given observation (word) being generated from a particular state (POS tag). 

- **Example in POS tagging**: 
  - For example, the word **"cat"** is most likely to be tagged as a **Noun (NN)**, and **"sleeps"** is most likely to be tagged as a **Verb (VBZ)**.
  - Emission probabilities specify how likely it is that a specific word corresponds to a particular POS tag.

- **Why We Need Emission Probabilities**:
  - Without emission probabilities, the algorithm wouldn’t be able to assess the likelihood of a word belonging to a specific POS tag. This is crucial for determining the best tag sequence.

---

### **Why You Need Both Transition and Emission Probabilities in POS Tagging:**

- **Transition probabilities** allow the algorithm to choose the most probable path of states (tags in the case of POS tagging).
  - For example, the model might evaluate the likelihood of a sequence like **DT → NN → VBZ**.

- **Emission probabilities** allow the algorithm to calculate the likelihood of observing specific words given a state.
  - For example, the model might evaluate the likelihood of the word **"cat"** being tagged as **NN**.

#### **Example**:

For a sentence like **"The cat sleeps"**, you’d need both transition and emission probabilities for the model to:

- Estimate the likelihood of transitions between POS tags:
  - E.g., how likely is it that a word tagged as **DT** (Determiner) is followed by a word tagged as **NN** (Noun), and how likely is it that a **NN** is followed by a **VBZ** (Verb, 3rd person singular)?
  
- Estimate the likelihood of each word being associated with a specific POS tag:
  - E.g., how likely is the word **"cat"** to be tagged as **NN** and **"sleeps"** to be tagged as **VBZ**?

---

### **Defining Probabilities**:

In practice, these probabilities are either:

1. **Manually Defined**:
   - Based on linguistic knowledge or a corpus of labeled data.
   - For small illustrative examples, these probabilities might be defined manually or with assumptions.

2. **Learned from Data**:
   - In real-world applications, these probabilities are **learned** by training a model on a labeled dataset (e.g., a large text corpus where words are already tagged with POS tags).
   - For example, using datasets like the **Penn Treebank** or similar corpora, transition and emission probabilities can be computed from the data.

---

### **Example of a Realistic Setup**:

1. **Transition Probability**: \( P(\text{tag}_t \mid \text{tag}_{t-1}) \)
   - Example: How likely is it that a word tagged as **NN** (Noun) is followed by a word tagged as **VBZ** (Verb, 3rd person singular)?

2. **Emission Probability**: \( P(\text{word}_t \mid \text{tag}_t) \)
   - Example: How likely is the word **"cat"** to be tagged as **NN**?

---



In [6]:
import numpy as np

# Define the transition and emission probabilities as dictionaries
transition_probs = {
    'DT': {'DT': 0.1, 'NN': 0.7, 'VBZ': 0.2},
    'NN': {'DT': 0.6, 'NN': 0.3, 'VBZ': 0.1},
    'VBZ': {'DT': 0.2, 'NN': 0.3, 'VBZ': 0.5}
}

emission_probs = {
    'DT': {'The': 0.9, 'cat': 0.1, 'sleeps': 0.05},
    'NN': {'The': 0.05, 'cat': 0.8, 'sleeps': 0.05},
    'VBZ': {'The': 0.05, 'cat': 0.1, 'sleeps': 0.9}
}

# Observations (words in the sentence)
observations = ['The', 'cat', 'sleeps']

# Hidden states (POS tags)
states = ['DT', 'NN', 'VBZ']

# Initialize the Viterbi matrix (stores the highest probabilities)
viterbi = {state: [0] * len(observations) for state in states}

# Initialize the backpointer matrix (for tracking the best path)
backpointer = {state: [None] * len(observations) for state in states}

# Step 1: Initialization (first observation)
for state in states:
    viterbi[state][0] = emission_probs[state].get(observations[0], 0) * 1  # Initial probability (P(start) = 1)

# Step 2: Recursion (for subsequent observations)
k = 2  # Keep top k most probable states
for t in range(1, len(observations)):
    word = observations[t]
    
    for current_state in states:
        state_probabilities = []
        
        for previous_state in states:
            transition_prob = transition_probs[previous_state].get(current_state, 0)
            emission_prob = emission_probs[current_state].get(word, 0)
            prob = viterbi[previous_state][t-1] * transition_prob * emission_prob
            state_probabilities.append((prob, previous_state))
        
        # Sort and keep the top k states
        state_probabilities.sort(reverse=True, key=lambda x: x[0])
        top_k_states = state_probabilities[:k]
        
        # Store the most probable path
        best_prob, best_state = top_k_states[0]
        viterbi[current_state][t] = best_prob
        backpointer[current_state][t] = best_state

# Step 3: Termination (backtrack to find the best path)
# Start from the final time step (last word in the observations)
best_final_state = max(viterbi, key=lambda state: viterbi[state][len(observations)-1])
best_path = [best_final_state]

# Backtrack to find the most probable state sequence
for t in range(len(observations)-1, 0, -1):
    best_final_state = backpointer[best_final_state][t]
    best_path.insert(0, best_final_state)

# Output the best path (POS tags sequence)
print("Best POS tag sequence:", best_path)


Best POS tag sequence: ['DT', 'NN', 'VBZ']
